In [1]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
filename = 'Energy_Indicators.xls'
energy = pd.read_excel(filename, header=None, skipfooter=1)

In [3]:
energy = energy.drop([0,1], axis=1)
energy = energy.iloc[18:245]


In [4]:
energy.columns = ['Country','Energy Supply','Energy Supply per Capita','% Renewable']
energy.index = np.arange(len(energy))

In [5]:
def changeEnergySupply(x):
   x = x*1000000
   return x


In [6]:
energy.loc[:, 'Energy Supply'] = energy['Energy Supply'].apply(changeEnergySupply)

In [7]:
energy['Energy Supply'].replace('...', np.NaN, inplace=True)
energy['Energy Supply per Capita'].replace('...', np.NaN, inplace=True)
energy['% Renewable'].replace('...', np.NaN, inplace=True)

In [8]:
def remove_other(cell_value):
    without_parentheses = re.sub(r'\([^)]*\)', '', cell_value)
    without_numbers = re.sub(r'\d', '', without_parentheses)
    return without_numbers

energy['Country'] = energy['Country'].apply(remove_other)
energy['Country'] = energy['Country'].str.strip()

In [9]:
replacement_dict = {
    'Republic of Korea': 'South Korea',
    'United States of America': 'United States',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'China, Hong Kong Special Administrative Region': 'Hong Kong'
}

energy['Country'] = energy['Country'].replace(replacement_dict) 


In [10]:
url_world_bank = 'API.csv'
GDP = pd.read_csv(url_world_bank, skiprows=3, dtype = None, encoding='utf-8')

In [11]:
GDP.loc[GDP['Country Name'] == 'Korea, Rep.', 'Country Name'] = 'South Korea'
GDP.loc[GDP['Country Name'] == 'Iran, Islamic Rep.', 'Country Name'] = 'Iran'
GDP.loc[GDP['Country Name'] == 'Hong Kong SAR, China', 'Country Name'] = 'Hong Kong'

GDP = GDP.rename(columns={'Country Name': 'Country'}) 

In [12]:
url_ScimEn = 'scimagojr country rank 1996-2022.xlsx'
ScimEn = pd.read_excel(url_ScimEn)

In [13]:
def answer_one():
    merged_df = ScimEn.merge(energy, on='Country')
    merged_df = merged_df.merge(GDP, on='Country')
    
    merged_df = merged_df.set_index('Country')
    
    merged_df = merged_df.loc[:,['Rank','Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index','Energy Supply', 'Energy Supply per Capita', '% Renewable','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015',]]
    return merged_df.iloc[0:15]



In [14]:
display(answer_one())

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,360468,358777,3947871,2705774,10.95,308,127191000000,93.0,19.75491,2.752119e+12,3.550328e+12,4.594337e+12,5.101691e+12,6.087192e+12,7.551545e+12,8.532186e+12,9.570471e+12,1.047562e+13,1.106157e+13
United States,2,199442,195042,3068926,881789,15.39,422,90838000000,286.0,11.57098,1.381559e+13,1.447423e+13,1.476986e+13,1.447806e+13,1.504896e+13,1.559973e+13,1.625397e+13,1.684319e+13,1.755068e+13,1.820602e+13
India,3,76103,74167,760964,280893,10.00,217,33195000000,26.0,14.96908,9.402599e+11,1.216736e+12,1.198895e+12,1.341888e+12,1.675616e+12,1.823052e+12,1.827638e+12,1.856721e+12,2.039126e+12,2.103588e+12
Japan,4,56249,55680,633294,136132,11.26,217,18984000000,149.0,10.23282,4.601663e+12,4.579750e+12,5.106679e+12,5.289494e+12,5.759072e+12,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12
United Kingdom,5,52572,51156,909276,151672,17.30,267,7920000000,124.0,10.60047,2.709978e+12,3.092996e+12,2.931684e+12,2.417566e+12,2.491397e+12,2.666403e+12,2.706341e+12,2.786315e+12,3.065223e+12,2.934858e+12
Germany,6,47781,46767,641717,133693,13.43,230,13261000000,165.0,17.90153,2.994704e+12,3.425578e+12,3.745264e+12,3.411261e+12,3.399668e+12,3.749315e+12,3.527143e+12,3.733805e+12,3.889093e+12,3.357586e+12
Russian Federation,7,43567,43290,175721,79765,4.03,103,30709000000,214.0,17.28868,9.899321e+11,1.299703e+12,1.660848e+12,1.222646e+12,1.524917e+12,2.045923e+12,2.208294e+12,2.292470e+12,2.059242e+12,1.363482e+12
Canada,8,39036,38276,787010,125333,20.16,263,10431000000,296.0,61.94543,1.319265e+12,1.468820e+12,1.552990e+12,1.374625e+12,1.617343e+12,1.793327e+12,1.828366e+12,1.846597e+12,1.805750e+12,1.556509e+12
Italy,9,35991,34424,529459,123042,14.71,192,6530000000,109.0,33.66723,1.949552e+12,2.213102e+12,2.408655e+12,2.199929e+12,2.136100e+12,2.294994e+12,2.086958e+12,2.141924e+12,2.162010e+12,1.836638e+12


In [15]:
def answer_two():
    Top15 = answer_one()
    columns_for_GDP = Top15.columns[10:20] 
    Top15['avgGDP'] = Top15[columns_for_GDP].mean(axis=1, skipna=True)
    
    Top15 = Top15.iloc[:, -1:]
    avgGDP = Top15.sort_values(by = 'avgGDP', ascending=False)
    return avgGDP    

In [16]:
display(answer_two())

,avgGDP
Country,
United States,1.570403e+13
China,6.927707e+12
Japan,5.239642e+12
Germany,3.523342e+12
United Kingdom,2.780276e+12
France,2.691337e+12
Italy,2.142986e+12
Brazil,1.988889e+12
Russian Federation,1.666746e+12


In [17]:
def answer_three():
    Top15 = answer_one()
    Country_list = answer_two()
    listIndex = Country_list.index
    listIndex = listIndex[5]
    
    
    change_avg = Top15.loc[listIndex]
    change_avg = change_avg.iloc[10] - change_avg.iloc[19]
    
    return listIndex, change_avg

In [18]:
result = answer_three()
print (f'ВВП {result[0]} змінилося за останні 10 років на : {result[1]} ')

ВВП France змінилося за останні 10 років на : -118652421857.7998 


In [19]:
def answer_four():
    Top15 = answer_one()
    
    Top15['relation'] = Top15['Self-citations']/Top15['Citations']
    
    max_relation = Top15['relation'].max()
    max_relation_index = Top15['relation'].idxmax()
    return max_relation_index, max_relation 

In [20]:
result = answer_four()

print(f"Країна: {result[0]}, відношення: {result[1]} ")


Країна: China, відношення: 0.6853754846599598 


In [21]:
def answer_five():
    Top15 = answer_one()
    
    Top15['population'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    
    population = Top15.sort_values(by='population', ascending=False)
    
    population2 = Top15.population[1] # Значення 2гої країни за оцінкою
    index = population.index[1]
    
    return population2, index, Top15

In [22]:
result = answer_five()

print(f"Країна: {result[1]}, населення: {result[0]} ")

Країна: India, населення: 317615384.61538464 


In [23]:
def answer_six():
    Top15 = answer_one()
    
    correl = Top15['Citations per document'].corr(Top15['Energy Supply per Capita'])
    return correl

In [24]:
print(f"Корелія: {answer_six()}")

Корелія: 0.3171912874561641


In [25]:
ContinentDict  = {
                  'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'
}


In [53]:
def answer_seven():
    Top15 = answer_one()
    df = answer_five()
    df = df[2]
    df = df[['population']]# населення країн
    
    Top15['Continent'] = Top15.index.map(ContinentDict)
    Top15 = Top15[['Continent']]
    
    Top15 = Top15.merge(df, on='Country')
    
    list_group = Top15.groupby('Continent')
    
    size = list_group.size()
    total_population = list_group['population'].sum()
    mean_population = list_group['population'].mean()
    std_population = list_group['population'].std()
    
    result_df = pd.DataFrame(
         {'size': size,
          'sum': total_population,
          'mean': mean_population,
          'std': std_population
         })
    
    return result_df


In [54]:
display(answer_seven())

,size,sum,mean,std
Continent,,,,
Asia,5,2898666386.6106,579733277.32212,6.790979e+08
Australia,1,23316017.316017,23316017.316017,NaN
Europe,6,457929667.216372,76321611.202729,3.464767e+07
North America,2,352855249.48025,176427624.740125,1.996696e+08
South America,1,205915254.237288,205915254.237288,NaN
